In [ ]:
# Deletes the first 5 rows from .txt file because they contain useless information.
# Then save it to a new file.

file = open('games_original.txt', "r")
lines = file.readlines()
file.close()

for i in range(5):
    del lines[0]

new_file = open("games.txt", "w+")

for line in lines:
    new_file.write(line)
new_file.close()

In [ ]:
# DO NOT WORRY THIS STEP TAKES A LOT OF TIME BECAUSE OF THE FILE SIZE(2.5GB).
# Reading in the data and converting it to a Pandas dataframe.

import pandas as pd

raw_data = pd.read_csv('games.txt', delimiter=" ### ", header=None)

column_names = ["#", "date", "result", "welo", "belo", "len", "date_c", "resu_c", "welo_c", "belo_c", "edate_c", "setup", "fen", "resu2_c", "oyrange", "bad_len", "DELETE THIS"]
data = pd.DataFrame(raw_data[0].str.split(" ").to_list(), columns=column_names)

data.drop(['DELETE THIS'], axis=1, inplace=True)

data = pd.concat([data, raw_data[1]], axis=1)
data = data.rename(columns = {1: 'moves'}, inplace = False)
data['len'] = data['len'].astype(int)
data.head(5)

In [ ]:
data.columns

In [ ]:
import numpy as np

# Data cleaning.
# Removing columns and rows that are useless to use.
data.drop(data[data["setup"] != "setup_false"].index, inplace=True)
data.drop(['resu_c', 'welo_c', 'belo_c', 'setup', 'fen', 'oyrange', 'bad_len', 'resu2_c', 'edate_c', 'date_c'], axis=1, inplace=True)
data.drop(data[data.result == "*"].index, inplace=True) # Remove all rows where the result is "*", meaning that the result is corrupted or missing.
data.drop(data[data.len < 4].index, inplace=True) # Removing all rows where the len is smaller than 4.

# Changing datatypes
data.welo = data.welo.replace("None", np.nan)
data.belo = data.belo.replace("None", np.nan)
data.welo = data.welo.astype("float")
data.belo = data.belo.astype("float")

data.head()

In [ ]:
rcParams["figure.figsize"] = 12, 12
#plt.scatter(data["welo"], data["belo"])
#plt.xlabel("White elo")
#plt.ylabel("Black elo")

import seaborn as sns

elo_ranges = [[0, 500], [501, 1000], [1001, 1500], [1501, 2000], [2001, 2500], [2501, 3000], ["None", "None"]]

# White elo ranges on x
# Black elo ranges on y
x_columns = [str(x[0]) + "-" + str(x[1]) for x in elo_ranges]
y_columns = [str(x[0]) + "-" + str(x[1]) for x in elo_ranges]
elo_clusters = pd.DataFrame(columns=x_columns, index=y_columns)
elo_clusters["0-500"]["501-1000"] = 1
elo_clusters
print(elo_clusters.columns)

dataset = data

for white_elo_range in elo_ranges:
    for black_elo_range in elo_ranges:
        white_elo = str(white_elo_range[0]) + "-" + str(white_elo_range[1])
        black_elo = str(black_elo_range[0]) + "-" + str(black_elo_range[1])
        min_elo_w = white_elo_range[0]
        max_elo_w = white_elo_range[1]
        min_elo_b = black_elo_range[0]
        max_elo_b = black_elo_range[1]
        if white_elo == "None-None":
            white_count = dataset.welo.isnull()
        else:
            white_count = (dataset["welo"] >= min_elo_w) & (dataset["welo"] < max_elo_w)
        
        if black_elo == "None-None":
            black_count = dataset.belo.isnull()
        else:
            black_count = (dataset["belo"] < max_elo_b) & (dataset["belo"] >= min_elo_b)
           
        count = len(dataset[white_count & black_count])
        
        elo_clusters[white_elo][black_elo] = count

elo_clusters


#rcParams['figure.figsize'] = 7, 7
#tab = pd.crosstab(mini["welo"],mini["belo"])
#sns.heatmap(tab, annot=True, fmt="d")
#plt.ylabel("Cluster")

In [ ]:
## Suure dataseti subset. Lihtsam jooksutada erinevaid algosid jne.
mini = data.head(1000)

In [ ]:
#MITTE KUSTUTADA
#PRIIT PÄRN TEGELEB SELLEGA
data_copy = data.copy(deep=True)
data_copy.drop(['date', 'welo', 'belo', 'len'], axis=1, inplace=True)
data_copy.drop(data_copy[data_copy.result == "1/2-1/2"].index, inplace=True) # Remove all rows where the game ended in a draw.
data_copy.head()

In [ ]:
#MITTE KUSTUTADA
#PRIIT PÄRN TEGELEB SELLEGA
winnersMoves = []
for index, row in data_copy.iterrows():
    if (row['result'] == "1-0"): # If the white player won.
        movesWithQueue = row['moves'].split(" ")
        moves = ""
        i = 0
        for moveWithQueue in movesWithQueue:
            if (i == 5):
                break
            if (moveWithQueue[0] == "W"):
                move = moveWithQueue.split(".")
                moves += " "
                moves += move[1]
                i += 1
        winnersMoves.append(moves)
    else: # If the black player won.
        movesWithQueue = row['moves'].split(" ")
        moves = ""
        j = 0
        for moveWithQueue in movesWithQueue:
            if (j == 5):
                break
            if (moveWithQueue[0] == "B"):
                move = moveWithQueue.split(".")
                moves += " "
                moves += move[1]
                j += 1
                
        winnersMoves.append(moves)

winnerMovesDataFrame = pd.DataFrame(data=winnersMoves, columns = ['Moves'])
winnerMovesDataFrame.head()

print(winnerMovesDataFrame["Moves"].value_counts())

In [ ]:
## JOOOOOOOOOSEP TEB 

from matplotlib import rcParams
import matplotlib.pyplot as plt

## Goal 4: Find out the main differences in moves according to elo.

rcParams["figure.figsize"] = 15, 30
subplot_index = 1
elo_ranges = [[0, 500], [501, 1000], [1001, 1500], [1501, 2000], [2001, 2500], [2501, 3000]]

dataset = mini

for elo_range in elo_ranges:
    min_elo = elo_range[0]
    max_elo = elo_range[1]
    elo = str(min_elo) + "-" + str(max_elo)
    print(elo)
    
    white_moves = []
    white_results = []
    black_moves = []
    black_results = []

    for index, row in dataset.iterrows():
        if row.welo >= min_elo and row.welo < max_elo:
            first_move = row.moves.split(" ")[0].split(".")[1] # white's move
            result = row.result
            white_moves.append(first_move)
            white_results.append(result)
        if row.belo >= min_elo and row.belo < max_elo:
            first_move = row.moves.split(" ")[1].split(".")[1] # black's move
            result = row.result
            black_moves.append(first_move)
            black_results.append(result)
    
    white_moves_df = pd.DataFrame({"result": white_results, "move": white_moves})
    white_moves = white_moves_df.move.unique()
    white_winrates = pd.DataFrame({"wins": [len(white_moves_df[(white_moves_df["move"]==move) & (white_moves_df["result"]=="1-0")]) for move in white_moves],
                             "losses": [len(white_moves_df[(white_moves_df["move"]==move) & (white_moves_df["result"]=="0-1")]) for move in white_moves],
                             "draws": [len(white_moves_df[(white_moves_df["move"]==move) & (white_moves_df["result"]=="1/2-1/2")]) for move in white_moves],
                                  "total_games": [len(white_moves_df[(white_moves_df["move"]==move)])for move in white_moves]},
                            index=white_moves)
    
    black_moves_df = pd.DataFrame({"result": black_results, "move": black_moves})
    black_moves = black_moves_df.move.unique()
    black_winrates = pd.DataFrame({"wins": [len(black_moves_df[(black_moves_df["move"]==move) & (black_moves_df["result"]=="1-0")])for move in black_moves],
                             "losses": [len(black_moves_df[(black_moves_df["move"]==move) & (black_moves_df["result"]=="0-1")])for move in black_moves],
                             "draws": [len(black_moves_df[(black_moves_df["move"]==move) & (black_moves_df["result"]=="1/2-1/2")])for move in black_moves],
                                  "total_games": [len(black_moves_df[(black_moves_df["move"]==move)])for move in black_moves]},
                            index=black_moves)
        
    # Plot for white
    width = 0.25
    bars1 = np.arange(len(white_moves))
    bars2 = [x + width for x in bars1]
    bars3 = [x + width for x in bars2]
    
    wins = [row.wins / row.total_games for idx,row in white_winrates.iterrows()]
    losses = [row.losses / row.total_games for idx,row in white_winrates.iterrows()]
    draws = [row.draws / row.total_games for idx,row in white_winrates.iterrows()]
    
    # Sketchy but maybe works
    if not wins:
        wins = [0]
    if not losses:
        losses = [0]
    if not draws:
        draws = [0]
        
    plt.subplot(6, 2, subplot_index)
    plt.bar(bars1, wins, width=width, label="wins")
    plt.bar(bars2, losses, width=width, label="losses")
    plt.bar(bars3, draws, width=width, label="draws")
    plt.title("white " + elo)
    plt.xticks([x + width for x in range(len(white_moves))], white_moves)
    subplot_index += 1
    plt.legend()
    
    # Plot for black
    width = 0.25
    bars1 = np.arange(len(black_moves))
    bars2 = [x + width for x in bars1]
    bars3 = [x + width for x in bars2]

    wins = [row.wins / row.total_games for idx,row in black_winrates.iterrows()]
    losses = [row.losses / row.total_games for idx,row in black_winrates.iterrows()]
    draws = [row.draws / row.total_games for idx,row in black_winrates.iterrows()]
    
    # Sketchy but maybe works
    if not wins:
        wins = [0]
    if not losses:
        losses = [0]
    if not draws:
        draws = [0]
        
    plt.subplot(6, 2, subplot_index)
    plt.bar(bars1, wins, width=width, label="wins")
    plt.bar(bars2, losses, width=width, label="losses")
    plt.bar(bars3, draws, width=width, label="draws")
    plt.title("black " + elo)
    plt.xticks([x + width for x in range(len(black_moves))], black_moves)
    subplot_index += 1
    plt.legend()

In [ ]:
print(black_winrates)

## Võiks välja jätta käigud, mida on tehtud vähem kui 5%

In [ ]:
rcParams["figure.figsize"] = 15, 30
subplot_index = 1
elo_ranges = [[0, 500], [501, 1000], [1001, 1500], [1501, 2000], [2001, 2500], [2501, 3000]]

for elo_range in elo_ranges:
    min_elo = elo_range[0]
    max_elo = elo_range[1]
    elo = str(min_elo) + "-" + str(max_elo)
    print(elo)
    
    white_moves = []
    white_results = []
    black_moves = []
    black_results = []

    for index, row in mini.iterrows():
        if row.welo >= min_elo and row.welo < max_elo:
            first5_moves = row.moves.split(" ")[0].split(".")[1] # white's move
            result = row.result
            white_moves.append(first5_moves)
            white_results.append(result)
        if row.belo >= min_elo and row.belo < max_elo:
            first5_moves = row.moves.split(" ")[1].split(".")[1] # black's move
            result = row.result
            black_moves.append(first5_moves)
            black_results.append(result)
    
    white_moves_df = pd.DataFrame({"result": white_results, "move": white_moves})
    white_moves = white_moves_df.move.unique()
    white_winrates = pd.DataFrame({"wins": [len(white_moves_df[(white_moves_df["move"]==move) & (white_moves_df["result"]=="1-0")]) for move in white_moves],
                             "losses": [len(white_moves_df[(white_moves_df["move"]==move) & (white_moves_df["result"]=="0-1")]) for move in white_moves],
                             "draws": [len(white_moves_df[(white_moves_df["move"]==move) & (white_moves_df["result"]=="1/2-1/2")]) for move in white_moves],
                                  "total_games": [len(white_moves_df[(white_moves_df["move"]==move)])for move in white_moves]},
                            index=white_moves)
    
    black_moves_df = pd.DataFrame({"result": black_results, "move": black_moves})
    black_moves = black_moves_df.move.unique()
    black_winrates = pd.DataFrame({"wins": [len(black_moves_df[(black_moves_df["move"]==move) & (black_moves_df["result"]=="1-0")])for move in black_moves],
                             "losses": [len(black_moves_df[(black_moves_df["move"]==move) & (black_moves_df["result"]=="0-1")])for move in black_moves],
                             "draws": [len(black_moves_df[(black_moves_df["move"]==move) & (black_moves_df["result"]=="1/2-1/2")])for move in black_moves],
                                  "total_games": [len(black_moves_df[(black_moves_df["move"]==move)])for move in black_moves]},
                            index=black_moves)
        
    # Plot for white
    width = 0.25
    bars1 = np.arange(len(white_moves))
    bars2 = [x + width for x in bars1]
    bars3 = [x + width for x in bars2]
    
    wins = [row.wins / row.total_games for idx,row in white_winrates.iterrows()]
    losses = [row.losses / row.total_games for idx,row in white_winrates.iterrows()]
    draws = [row.draws / row.total_games for idx,row in white_winrates.iterrows()]
    
    # Sketchy but maybe works
    if not wins:
        wins = [0]
    if not losses:
        losses = [0]
    if not draws:
        draws = [0]
        
    plt.subplot(6, 2, subplot_index)
    plt.bar(bars1, wins, width=width, label="wins")
    plt.bar(bars2, losses, width=width, label="losses")
    plt.bar(bars3, draws, width=width, label="draws")
    plt.title("white " + elo)
    plt.xticks([x + width for x in range(len(white_moves))], white_moves)
    subplot_index += 1
    plt.legend()
    
    # Plot for black
    width = 0.25
    bars1 = np.arange(len(black_moves))
    bars2 = [x + width for x in bars1]
    bars3 = [x + width for x in bars2]

    wins = [row.wins / row.total_games for idx,row in black_winrates.iterrows()]
    losses = [row.losses / row.total_games for idx,row in black_winrates.iterrows()]
    draws = [row.draws / row.total_games for idx,row in black_winrates.iterrows()]
    
    # Sketchy but maybe works
    if not wins:
        wins = [0]
    if not losses:
        losses = [0]
    if not draws:
        draws = [0]
        
    plt.subplot(6, 2, subplot_index)
    plt.bar(bars1, wins, width=width, label="wins")
    plt.bar(bars2, losses, width=width, label="losses")
    plt.bar(bars3, draws, width=width, label="draws")
    plt.title("black " + elo)
    plt.xticks([x + width for x in range(len(black_moves))], black_moves)
    subplot_index += 1
    plt.legend()

In [ ]:
moves = []
for idx, row in mini.iterrows():
    current_game = []
    for move in row["moves"].split(" "):
        current_game.append(move.split(".")[1])
    moves.append(current_game)

print(moves[0])